## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
import numpy as np
import os
import struct
import pandas as pd
import time
from time import process_time
from itertools import chain 
import gc

In [3]:
dataset = 'smallSift'

if dataset == 'sift':
  url = "ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz"
  paths =  '/tmp/sift.tar.gz' 
  trainPath = 'sift/sift_base.fvecs'
  queryPath = 'sift/sift_query.fvecs'
  groundPath = 'sift/sift_groundtruth.ivecs'
  
if dataset == 'smallSift':
  url = 'ftp://ftp.irisa.fr/local/texmex/corpus/siftsmall.tar.gz'
  paths =  '/tmp/siftsmall.tar.gz' 
  trainPath = 'siftsmall/siftsmall_base.fvecs'
  queryPath = 'siftsmall/siftsmall_query.fvecs'
  groundPath = 'siftsmall/siftsmall_groundtruth.ivecs'
  

In [4]:
import urllib 
urllib.request.urlretrieve(url, paths)

Out[20]: ('/tmp/siftsmall.tar.gz', <email.message.Message at 0x7f74f431b5d0>)

In [5]:
def get_zippedFvecs(pathToGz,memeber):
    import tarfile
    fn = pathToGz
    import struct
    import numpy as np
    t = tarfile.open(fn, 'r:gz') 
    
    
    m = t.getmember(memeber)
    file = t.extractfile(m)
    fileSize = m.size
    #file =  open(path,'rb')
    #first 4 bytes of every vector indicate number od dimensions 
    numOfDimensions = struct.unpack('i', file.read(4))[0]
    #each vector has 4 bytes (float is 32 bits) * numberOfDimensions
    #plus 4 bytes long indicator as mentioned  
    numOfVectors = (int) (fileSize / (4 + 4*numOfDimensions))
    #init empty list for vectors
    #vectors = []
    vectors = np.zeros((numOfVectors,numOfDimensions))
    #return to the beginning
    file.seek(0)
    for vecotr in range(numOfVectors):
        file.read(4) #go trough indicator of dimensions
        #vectors.append(struct.unpack('f' * numOfDimensions, file.read(4*numOfDimensions)))
        vectors[vecotr] = struct.unpack('f' * numOfDimensions, file.read(4*numOfDimensions))
    file.close()
    return vectors

In [6]:

def get_zippedIvecs(pathToGz,memeber):
    import tarfile
    fn = pathToGz
    import struct
    import numpy as np
    t = tarfile.open(fn, 'r:gz') 
    
    
    m = t.getmember(memeber)
    file = t.extractfile(m)
    fileSize = m.size
    #file =  open(path,'rb')
    #first 4 bytes of every vector indicate number od dimensions 
    numOfDimensions = struct.unpack('i', file.read(4))[0]
    #each vector has 4 bytes (float is 32 bits) * numberOfDimensions
    #plus 4 bytes long indicator as mentioned  
    numOfVectors = (int) (fileSize / (4 + 4*numOfDimensions))
    #init empty list for vectors
    #vectors = []
    vectors = np.zeros((numOfVectors,numOfDimensions), int)
    #return to the beginning
    file.seek(0)
    for vecotr in range(numOfVectors):
        file.read(4) #go trough indicator of dimensions
        #vectors.append(struct.unpack('f' * numOfDimensions, file.read(4*numOfDimensions)))
        vectors[vecotr] = struct.unpack('i' * numOfDimensions, file.read(4*numOfDimensions))
    file.close()
    return vectors


In [7]:
def returnRecall(result, test):
    numOfTrueNeighbours = []
    #for every result vector we check how many right neighbours were identified
    for i in range(result.shape[0]):
        numTN = len(set(result[i].tolist()) & set(test[i].tolist()))
        numOfTrueNeighbours.append(numTN)
        recall = sum(numOfTrueNeighbours) /test.size
    return recall

In [8]:
def fillIfNotAllAreFound(result):
    for i in range(len(result)):
        if len(result[i]) < 100: 
            result[i].extend((100-len(result[i])) *[-1]) 
    return result

In [9]:
train = get_zippedFvecs(paths,trainPath )
#there is 100 querry ponts
query = get_zippedFvecs(paths,queryPath )
#there is index number of 100 nearset n. for each querry point
groundTruth = get_zippedIvecs(paths,groundPath )

In [10]:
rm /tmp/sift.tar.gz

In [11]:
algorithm = []
construciotnTimes=[]
searchTimes=[]
reacll = []
k = 100
avgdistances = []
constructionClocks = []
searchClocks = []
clockAlg = []

In [12]:
#Annoy
from annoy import AnnoyIndex
for trs in [5,15,30,60,80]:
    
    f = train.shape[1]
    t = AnnoyIndex(f, 'euclidean')
    
    startClock= time.clock()
    startTime = process_time()
    for i in range(train.shape[0]):
        t.add_item(i,train[i])
    t.build(trs)
    end_time = process_time()
    constructionTime = end_time - startTime
    endClock = time.clock()
    constructionClock= endClock - startClock
    
    
    rez = []
    dist = []
    startClock = time.clock()
    startTime = process_time()
    for q in query:
        res,d = t.get_nns_by_vector(q, 100, include_distances=True)
        rez.append(res)
        dist.append(d)
        #result.append(t.get_nns_by_vector(q, 100, include_distances=True))
    end_time = process_time()
    searchTime = end_time - startTime
    endClock = time.clock()
    searchClock= endClock - startClock
    
        
    result = fillIfNotAllAreFound(rez)
    
    result = np.asanyarray(result)
    annoyRecall = returnRecall(result, groundTruth)  
    avgDist = np.mean(list(chain.from_iterable(dist)))
    
    reacll.append(annoyRecall)
    algorithm.append('Annoy-trees-'+str(trs))
    construciotnTimes.append(constructionTime)
    searchTimes.append(searchTime)
    avgdistances.append(avgDist)
    searchClocks.append(searchClock)
    constructionClocks.append(constructionClock)
    clockAlg.append('Annoy-trees-'+str(trs))
    t.save('annoyIndex90.ann')
    del t
    del rez
    del dist
    del result
    gc.collect()
#______________________________________________#


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3193702701822154> in <module> 
 3 for trs in [ 5 , 15 , 30 , 60 , 80 ] : 
 4 
 ----> 5 f = train . shape [ 1 ] 
 6 t = AnnoyIndex ( f , 'euclidean' ) 
 7 

 NameError : name 'train' is not defined

In [13]:
import nmslib
for MMAX in [5,8,15,30,38]:
    hnsw = nmslib.init(method='hnsw', space='l2')
    
    startClock = time.clock()
    startTime = process_time()
    hnsw.addDataPointBatch(train)
    hnsw.createIndex({'delaunay_type':1, 'M':MMAX})
    end_time = process_time()
    constructionTime = end_time - startTime
    endClock = time.clock()
    constructionClock= endClock - startClock
    
    
    
    # get all nearest neighbours for all the datapoint
    # using a pool of 4 threads to compute
    startClock = time.clock()
    startTime = process_time()
    neighbours = hnsw.knnQueryBatch(query, k=100, num_threads=2)
    end_time = process_time()
    searchTime = end_time - startTime
    endClock = time.clock()
    searchClock= endClock - startClock
    
    rez =[]
    dist =[]
    for i in neighbours:
        rez.append(list(i[0]))
        dist.append(list(i[1]))
    
    result = fillIfNotAllAreFound(rez)
      
    result = np.array(rez)
    hnswRecall = returnRecall(result, groundTruth)
    avgDist = np.mean(np.sqrt(list(chain.from_iterable(dist))))
    
    reacll.append(hnswRecall)
    algorithm.append('HNSW-M-'+str(MMAX))
    construciotnTimes.append(constructionTime)
    searchTimes.append(searchTime)
    avgdistances.append(avgDist)
    constructionClocks.append(constructionClock)
    searchClocks.append(searchClock)
    clockAlg.append('HNSW-M-'+str(MMAX))
    
    del hnsw
    del rez
    del dist
    del result
    del neighbours
    gc.collect()


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3193702701822153> in <module> 
 5 startClock = time . clock ( ) 
 6 startTime = process_time ( ) 
 ----> 7 hnsw . addDataPointBatch ( train ) 
 8 hnsw . createIndex ( { 'delaunay_type' : 1 , 'M' : MMAX } ) 
 9 end_time = process_time ( ) 

 NameError : name 'train' is not defined

In [14]:
import nmslib
vptree = nmslib.init(method='vptree', space='l2')

startClock = time.clock()
startTime = process_time()
vptree.addDataPointBatch(train)
vptree.createIndex({'bucketSize' : 10000,'selectPivotAttempts':10})
end_time = process_time()
constructionTime = end_time - startTime
endClock = time.clock()
constructionClock= endClock - startClock

# get all nearest neighbours for all the datapoint
# using a pool of 4 threads to compute
for maxLeave in [2,10,20,25,30]:
  
    vptree.setQueryTimeParams({'maxLeavesToVisit':maxLeave,'alphaLeft':1.1,'alphaRight':1.1})
    startClock = time.clock()
    startTime = process_time()
    neighbours = vptree.knnQueryBatch(query,k=100, num_threads=2 )
    end_time = process_time()
    searchTime = end_time - startTime
    endClock = time.clock()
    searchClock= endClock - startClock
    
    
    rez =[]
    dist = []
    for i in neighbours:
        rez.append(list(i[0]))
        dist.append(list(i[1]))
        
    rez = fillIfNotAllAreFound(rez)    
    
    result = np.asanyarray(rez)
    
    vptreeRecall = returnRecall(result, groundTruth)
    avgDist = np.mean(list(chain.from_iterable(dist)))
    
    reacll.append(vptreeRecall)
    algorithm.append('vp-Tree-10k-mL'+str(maxLeave))
    #algorithm.append('vp-Tree-maxLeaves'+str(maxLeaves))
    construciotnTimes.append(constructionTime)
    searchTimes.append(searchTime)
    constructionClocks.append(constructionClock)
    searchClocks.append(searchClock)
    avgdistances.append(avgDist)
    del rez
    del dist
    del result
    gc.collect()

#vptree.saveIndex('vptreeIndex.ann')    
del vptree
gc.collect()



--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3193702701822151> in <module> 
 4 startClock = time . clock ( ) 
 5 startTime = process_time ( ) 
 ----> 6 vptree . addDataPointBatch ( train ) 
 7 vptree . createIndex ( { 'bucketSize' : 10000 , 'selectPivotAttempts' : 10 } ) 
 8 end_time = process_time ( ) 

 NameError : name 'train' is not defined

In [16]:
compareResults = pd.DataFrame({ 'algorithm':algorithm, 'constructionTime':construciotnTimes, 'searchTime':searchTimes,'recall':reacll,'avgDistance':avgdistances, 'constructionClocks':constructionClocks,'searchClocks':searchClocks})

In [17]:
compareResults

,algorithm,constructionTime,searchTime,recall,avgDistance,constructionClocks,searchClocks


In [18]:
display(compareResults)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-913127357586025> in <module> 
 ----> 1 display ( compareResults ) 

 /local_disk0/tmp/1598775860123-0/PythonShell.py in display (self, input, *args, **kwargs) 
 1095 if self . sparkSession is None : 
 1096 raise Exception ( 'SparkSession is required for display(pandas.DataFrame).' ) 
 -> 1097 self . display ( self . sparkSession . createDataFrame ( input ) ) 
 1098 elif type ( input ) . __module__ == 'databricks.koalas.frame' and type ( input ) . __name__ == 'DataFrame' : 
 1099 index_col = kwargs . get ( 'index_col' ) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 650 # Create a DataFrame from pandas DataFrame. 
 651 return super(SparkSession, self).createDataFrame(
 --> 652 data, schema, samplingRatio, verifySchema)
 653 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 
 654 

 /databricks/spark/python/pyspark/sql/pandas/conversion.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 300 raise 
 301 data = self . _convert_from_pandas ( data , schema , timezone ) 
 --> 302 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 
 303 
 304 def _convert_from_pandas ( self , pdf , schema , timezone ) : 

 /databricks/spark/python/pyspark/sql/session.py in _create_dataframe (self, data, schema, samplingRatio, verifySchema) 
 683 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 684 else : 
 --> 685 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 686 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 687 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 468 write temp files . 
 469 """
 --> 470 data , schema = self . _wrap_data_schema ( data , schema ) 
 471 return self . _sc . parallelize ( data ) , schema
 472 

 /databricks/spark/python/pyspark/sql/session.py in _wrap_data_schema (self, data, schema) 
 447 
 448 if schema is None or isinstance ( schema , ( list , tuple ) ) : 
 --> 449 struct = self . _inferSchemaFromList ( data , names = schema ) 
 450 converter = _create_converter ( struct ) 
 451 data = map ( converter , data ) 

 /databricks/spark/python/pyspark/sql/session.py in _inferSchemaFromList (self, data, names) 
 378 """
 379 if not data : 
 --> 380 raise ValueError ( "can not infer schema from empty dataset" ) 
 381 first = data [ 0 ] 
 382 if type ( first ) is dict : 

 ValueError : can not infer schema from empty dataset

In [19]:
compareResults.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/compareResults.csv")

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-913127357586026> in <module> 
 ----> 1 compareResults . coalesce ( 1 ) . write . format ( "com.databricks.spark.csv" ) . option ( "header" , "true" ) . save ( "dbfs:/FileStore/df/compareResults.csv" ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/generic.py in __getattr__ (self, name) 
 5272 if self . _info_axis . _can_hold_identifiers_and_holds_name ( name ) : 
 5273 return self [ name ] 
 -> 5274 return object . __getattribute__ ( self , name ) 
 5275 
 5276 def __setattr__ ( self , name : str , value ) -> None : 

 AttributeError : 'DataFrame' object has no attribute 'coalesce'

In [20]:
import nmslib
vptree = nmslib.init(method='vptree', space='l2')

startClock = time.clock()
startTime = process_time()
vptree.addDataPointBatch(train)
vptree.createIndex({'bucketSize' : 10000,'selectPivotAttempts':10})
end_time = process_time()
constructionTime = end_time - startTime
endClock = time.clock()
constructionClock= endClock - startClock

# get all nearest neighbours for all the datapoint
# using a pool of 4 threads to compute
for maxLeave in [2,10,20,25,30]:
  
    vptree.setQueryTimeParams({'maxLeavesToVisit':maxLeave,'alphaLeft':1.1,'alphaRight':1.1})
    startClock = time.clock()
    startTime = process_time()
    neighbours = vptree.knnQueryBatch(query,k=100, num_threads=2 )
    end_time = process_time()
    searchTime = end_time - startTime
    endClock = time.clock()
    searchClock= endClock - startClock
    
    
    rez =[]
    dist = []
    for i in neighbours:
        rez.append(list(i[0]))
        dist.append(list(i[1]))
        
    rez = fillIfNotAllAreFound(rez)    
    
    result = np.asanyarray(rez)
    
    vptreeRecall = returnRecall(result, groundTruth)
    avgDist = np.mean(list(chain.from_iterable(dist)))
    
    reacll.append(vptreeRecall)
    algorithm.append('vp-Tree-10k-mL'+str(maxLeave))
    #algorithm.append('vp-Tree-maxLeaves'+str(maxLeaves))
    construciotnTimes.append(constructionTime)
    searchTimes.append(searchTime)
    constructionClocks.append(constructionClock)
    searchClocks.append(searchClock)
    avgdistances.append(avgDist)
    del rez
    del dist
    del result
    gc.collect()

#vptree.saveIndex('vptreeIndex.ann')    
del vptree
gc.collect()


#HNSW da bi islo po redu
#______________________________________________#
import nmslib
for MMAX in [5,8,15,30,38]:
    hnsw = nmslib.init(method='hnsw', space='l2')
    
    startClock = time.clock()
    startTime = process_time()
    hnsw.addDataPointBatch(train)
    hnsw.createIndex({'delaunay_type':1, 'M':MMAX})
    end_time = process_time()
    constructionTime = end_time - startTime
    endClock = time.clock()
    constructionClock= endClock - startClock
    
    
    
    # get all nearest neighbours for all the datapoint
    # using a pool of 4 threads to compute
    startClock = time.clock()
    startTime = process_time()
    neighbours = hnsw.knnQueryBatch(query, k=100, num_threads=2)
    end_time = process_time()
    searchTime = end_time - startTime
    endClock = time.clock()
    searchClock= endClock - startClock
    
    rez =[]
    dist =[]
    for i in neighbours:
        rez.append(list(i[0]))
        dist.append(list(i[1]))
    
    result = fillIfNotAllAreFound(rez)
      
    result = np.array(rez)
    hnswRecall = returnRecall(result, groundTruth)
    avgDist = np.mean(np.sqrt(list(chain.from_iterable(dist))))
    
    reacll.append(hnswRecall)
    algorithm.append('HNSW-M-'+str(MMAX))
    construciotnTimes.append(constructionTime)
    searchTimes.append(searchTime)
    avgdistances.append(avgDist)
    constructionClocks.append(constructionClock)
    searchClocks.append(searchClock)
    clockAlg.append('HNSW-M-'+str(MMAX))
    
    del hnsw
    del rez
    del dist
    del result
    del neighbours
    gc.collect()

#______________________________________________#    
#annoy
#Annoy
from annoy import AnnoyIndex
for trs in [5,15,30,60,80]:
    
    f = train.shape[1]
    t = AnnoyIndex(f, 'euclidean')
    
    startClock= time.clock()
    startTime = process_time()
    for i in range(train.shape[0]):
        t.add_item(i,train[i])
    t.build(trs)
    end_time = process_time()
    constructionTime = end_time - startTime
    endClock = time.clock()
    constructionClock= endClock - startClock
    
    
    rez = []
    dist = []
    startClock = time.clock()
    startTime = process_time()
    for q in query:
        res,d = t.get_nns_by_vector(q, 100, include_distances=True)
        rez.append(res)
        dist.append(d)
        #result.append(t.get_nns_by_vector(q, 100, include_distances=True))
    end_time = process_time()
    searchTime = end_time - startTime
    endClock = time.clock()
    searchClock= endClock - startClock
    
        
    result = fillIfNotAllAreFound(rez)
    
    result = np.asanyarray(result)
    annoyRecall = returnRecall(result, groundTruth)  
    avgDist = np.mean(list(chain.from_iterable(dist)))
    
    reacll.append(annoyRecall)
    algorithm.append('Annoy-trees-'+str(trs))
    construciotnTimes.append(constructionTime)
    searchTimes.append(searchTime)
    avgdistances.append(avgDist)
    searchClocks.append(searchClock)
    constructionClocks.append(constructionClock)
    clockAlg.append('Annoy-trees-'+str(trs))
    t.save('annoyIndex90.ann')
    del t
    del rez
    del dist
    del result
    gc.collect()
#______________________________________________#




--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1518425739103876> in <module> 
 4 startClock = time . clock ( ) 
 5 startTime = process_time ( ) 
 ----> 6 vptree . addDataPointBatch ( train ) 
 7 vptree . createIndex ( { 'bucketSize' : 10000 , 'selectPivotAttempts' : 10 } ) 
 8 end_time = process_time ( ) 

 NameError : name 'train' is not defined